In [ ]:
# for code to work download GTFS file from "https://transitfeeds.com/p/ptv/497" and unzip to "GTFS_single" directory
# rename files from txt to csv
# Vic governemnet website data is available at "https://discover.data.vic.gov.au/dataset/ptv-timetable-and-geographic-information-2015-gtfs"
# Vic data is in 10 separate zip files and may need to be combined
# GTFS info is avaialble at: "https://developers.google.com/transit/gtfs/reference"
# Melbourne suburb names were obtained from "https://www.homely.com.au/find-suburb-by-region/melbourne-greater-victoria"
#suburbs names were copied and pasted into a csv (greater_melbourne_suburbs.csv) file with "Suburb" column header

In [131]:
# Dependencies
import requests
import json
import hvplot.pandas
import pandas as pd
import requests

# Import the API key
from config import geoapify_key    
    

In [199]:
# Load the CSV file
calendar_df = pd.read_csv("GTFS_single/calendar.csv")
trips_df = pd.read_csv("GTFS_single/trips.csv")
stop_times_df = pd.read_csv("GTFS_single/stop_times.csv")
stops_df = pd.read_csv("GTFS_single/stops.csv")
# add extra column to stops file with suburb name extracted
stops_df["suburb"] = stops_df['stop_name'].str.split("(").str[1]
stops_df["suburb"] = stops_df['suburb'].str.split(")").str[0]
gms_df = pd.read_csv("greater_melbourne_suburbs.csv")

# Create a weekday only (single day) calendare dataframe (limit the data we work with)
# This is the base line for all calculations
weekday_df = calendar_df.loc[(calendar_df["wednesday"]==1) & (calendar_df["start_date"] <= 20230112) & (calendar_df["end_date"]>= 20230105)]

# test print of suburbs
print(gms_df)


            Suburbs
0        Abbotsford
1        Aberfeldie
2           Aintree
3      Airport West
4         Albanvale
..              ...
518  Yarra Junction
519       Yarrambat
520      Yarraville
521       Yellingbo
522          Yering

[523 rows x 1 columns]


In [206]:
# create a melbourne only stop list
melbourne = gms_df["Suburbs"]
melbourne_stops_wd_df = pd.DataFrame()
for i in melbourne:
#     print(i)
    stops_suburbs_wd_df = stops_df.loc[stops_df["suburb"] ==  i]
#     print(stops_suburbs_wd_df)
    melbourne_stops_wd_df = pd.concat([melbourne_stops_wd_df, stops_suburbs_wd_df], axis=0)

print(melbourne_stops_wd_df)

       stop_id                               stop_name   stop_lat    stop_lon  \
360        103   Nicholson St/Johnston St (Abbotsford) -37.800265  144.998067   
529        105  Trenerry Cres/Johnston St (Abbotsford) -37.800484  145.000378   
608        106      Clarke St/Johnston St (Abbotsford) -37.800626  145.001817   
1139      1129     Langridge St/Hoddle St (Abbotsford) -37.806948  144.992014   
1146      1130         Gipps St/Hoddle St (Abbotsford) -37.804764  144.992380   
...        ...                                     ...        ...         ...   
26760     8991      Geelong Rd/Angliss St (Yarraville) -37.812253  144.866830   
26761     8992   Somerville Rd/Geelong Rd (Yarraville) -37.811559  144.868792   
26762     8993        Simpson St/Fehon St (Yarraville) -37.816919  144.887188   
26763     8995          Hance St/Fehon St (Yarraville) -37.820432  144.886611   
26764     8996       Powell St/Francis St (Yarraville) -37.820499  144.884848   

           suburb  
360    

In [146]:
# extracting data from trips.csv to get trip_id from Weekday (single day) calendar
service_id_wd = weekday_df["service_id"]
# print(service_id_wd)
trips_extract_wd_df = pd.DataFrame()
# print(trips_extract_wd_df)
service_id
for i in service_id_wd:
#     print(i)
    trip_service_id_wd_df = trips_df.loc[trips_df["service_id"] ==  i]
#     print(trip_service_id_wd_df)
    trips_extract_wd_df = pd.concat([trips_extract_wd_df, trip_service_id_wd_df], axis=0)
#     trips_extract_df = trips_extract_df.append(trip_service_id_df)
print(trips_extract_wd_df)

             route_id service_id                    trip_id  \
16      1-Ech-D-mjp-1       1_T0  9601.T0.1-Ech-D-mjp-1.5.R   
17      1-Ech-D-mjp-1       1_T0  9602.T0.1-Ech-D-mjp-1.5.R   
18      1-Ech-D-mjp-1       1_T0  9603.T0.1-Ech-D-mjp-1.5.R   
19      1-Ech-D-mjp-1       1_T0  9604.T0.1-Ech-D-mjp-1.6.R   
20      1-Ech-D-mjp-1       1_T0  9825.T0.1-Ech-D-mjp-1.2.H   
...               ...        ...                        ...   
254874    6-R53-mjp-1    6_T0+pr    6341.T0.6-R53-mjp-1.1.H   
247018    6-C27-mjp-1  6_T0+pr_1     850.T0.6-C27-mjp-1.1.H   
247019    6-C27-mjp-1  6_T0+pr_1     851.T0.6-C27-mjp-1.2.R   
241153    6-89x-mjp-1    6_T0+pt    8863.T0.6-89x-mjp-1.1.H   
241154    6-89x-mjp-1    6_T0+pt    8864.T0.6-89x-mjp-1.1.H   

                 shape_id trip_headsign  direction_id  
16      1-Ech-D-mjp-1.5.R     Melbourne             1  
17      1-Ech-D-mjp-1.5.R     Melbourne             1  
18      1-Ech-D-mjp-1.5.R     Melbourne             1  
19      1-Ech-D-mjp

In [161]:
# just checking  original file length vs reduced data - this is not needed for the code
unique_trip_id = trips_extract_wd_df["trip_id"].unique()
print(len(unique_trip_id))
print(len(trips_df))

# Save the file so we do not have to extract again 
trips_extract_wd_df.to_csv("trips_extract_wd_df.csv")
      

37199
256966


In [148]:
# extracting stop_id from stop_times.csv - drilling  further linking the trips file
trip_id_wd = trips_extract_wd_df["trip_id"]
trip_id_wd
# print(stop_times_df.head())
stops_extract_wd_df = pd.DataFrame()
# service_id
for i in trip_id_wd:
#     print(i)
    stop_times_trip_id_wd_df = stop_times_df.loc[stop_times_df["trip_id"] ==  i]
#     print(stop_times_trip_id_wd_df.head())
    stops_extract_wd_df = pd.concat([stops_extract_wd_df, stop_times_trip_id_wd_df], axis=0)
print(stops_extract_wd_df)

                           trip_id arrival_time departure_time  stop_id  \
77235    9601.T0.1-Ech-D-mjp-1.5.R     11:04:00       11:04:00    20310   
77236    9601.T0.1-Ech-D-mjp-1.5.R     11:27:00       11:27:00    20338   
77237    9601.T0.1-Ech-D-mjp-1.5.R     11:42:00       11:42:00    20311   
77238    9601.T0.1-Ech-D-mjp-1.5.R     11:56:00       11:56:00    51809   
77239    9601.T0.1-Ech-D-mjp-1.5.R     12:04:00       12:04:00    49296   
...                            ...          ...            ...      ...   
9187493    8864.T0.6-89x-mjp-1.1.H     16:00:00       16:00:00    17148   
9187494    8864.T0.6-89x-mjp-1.1.H     16:11:12       16:11:12    22939   
9187495    8864.T0.6-89x-mjp-1.1.H     16:15:00       16:15:00    22940   
9187496    8864.T0.6-89x-mjp-1.1.H     16:18:53       16:18:53    48661   
9187497    8864.T0.6-89x-mjp-1.1.H     16:30:00       16:30:00    17148   

         stop_sequence  stop_headsign  pickup_type  drop_off_type  \
77235                1        

In [149]:
# Save the file so we do not have to extract again - extraction took overnight
stops_extract_wd_df.to_csv("stops_extract_wd_df.csv")

In [207]:
# Merge two dataframes using an inner join Adding stop names and GPS coordinates and suburb to trips
merge_df = pd.merge(stops_extract_wd_df, melbourne_stops_wd_df, on="stop_id")
merge_df.head

<bound method NDFrame.head of                            trip_id arrival_time departure_time  stop_id  \
0        9601.T0.1-Ech-D-mjp-1.5.R     13:52:00       13:52:00    19998   
1        9602.T0.1-Ech-D-mjp-1.5.R     17:52:00       17:52:00    19998   
2        9603.T0.1-Ech-D-mjp-1.5.R     10:11:00       10:11:00    19998   
3        9825.T0.1-Ech-D-mjp-1.2.H     07:32:00       07:32:00    19998   
4        9826.T0.1-Ech-D-mjp-1.4.H     15:37:00       15:37:00    19998   
...                            ...          ...            ...      ...   
1114438    2038.T0.6-w28-mjp-1.1.H     18:24:00       18:24:00    45031   
1114439    2057.T0.6-w41-mjp-1.1.H     17:09:00       17:09:00    45031   
1114440    2044.T0.6-w33-mjp-1.1.H     06:49:00       06:49:00    21465   
1114441    2056.T0.6-w40-mjp-1.1.H     17:54:00       17:54:00    45051   
1114442    2057.T0.6-w41-mjp-1.1.H     16:58:00       16:58:00    45051   

         stop_sequence  stop_headsign  pickup_type  drop_off_type  \


In [165]:
# save the file so we do not have to extract again
merge_df.to_csv("merge_df.csv")

In [214]:
# Create alist of suburbs and number of trips for a single day
# NOTE THIS STILL NEEDS TO  BE CLEANED ALL COLUMNS WILL STATE THE NUMBER OF  TRIPS IN A SUBURB
# TO RENAME FIRST COLUMN AND DELETE ALL OTHERS

suburb_group_df = merge_df.groupby(["suburb"])
suburb_group_df.count()
# Save the file - this is the end data for other cross rferences
suburb_group_df.count().to_csv("suburb_group_df.csv")

In [163]:
# random check to  ensure that the timetable at stop is accurate.
# I checked 2 stops near my house with 1 and 2 bus services at each stop. number of services were accurate
stop_check_df = merge_df.loc[merge_df["stop_id"] == 6210]
stop_check_df.to_csv("6210.csv")